In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet169, DenseNet121
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras.backend as K
from sklearn.metrics import roc_auc_score, cohen_kappa_score
import cv2
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm
import csv
import math
from keras.callbacks import LearningRateScheduler
%matplotlib inline

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
     self.val_f1s = []
     self.val_recalls = []
     self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
     val_predict = (np.asarray(self.model.predict(X_test_feats))).round()
     val_targ = Y_test
#      print(val_targ.shape)   
#      print(val_targ)
#      print(val_predict.shape)   
#      print(val_predict)   
     targ = []
     pred = []
     for x in val_targ:
        targ.append(np.argmax(x))
     for x in val_predict:
        pred.append(np.argmax(x))
#      print(targ)
#      print(pred)
        
     _val_f1 = f1_score(targ, pred)
     
     _val_recall = recall_score(targ, pred)
     _val_precision = precision_score(targ, pred)
     self.val_f1s.append(_val_f1)
     self.val_recalls.append(_val_recall)
     self.val_precisions.append(_val_precision)
     print("— val_f1: {0} — val_precision: {1} — val_recall {2} ".format(_val_f1, _val_precision, _val_recall))
     return

metrics = Metrics()

In [ ]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,validation_data):
#         self.x = training_data[0]
#         self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
#         y_pred = self.model.predict(self.x)
#         roc = roc_auc_score(self.y, y_pred)
        if(epoch%1==0):
            print("Calc Roc")
            all_rocs = []
            y_pred_val = self.model.predict(X_test,verbose=1)
            #print ("Intitial y_pred:{0}".format(y_pred_val))
            for d in range(2):
                try:
                    roc_val = roc_auc_score(Y_test[:,d], y_pred_val[:,d])
                    all_rocs.append(roc_val)
                except:
                    pass
            all_rocs = np.array(all_rocs)
            mean_roc = np.mean(all_rocs)
            
            Y_test_kappa=[np.argmax(i) for i in Y_test]
            y_pred_kappa=[np.argmax(i) for i in y_pred_val]
            try:
                score = f1_score(Y_test_kappa, np.around(y_pred_kappa))
                f1s = score
            except:
                    print(global_cls[d])
            avg_f1 = f1s
            
            kappa_score = cohen_kappa_score(Y_test_kappa, y_pred_kappa)            
            
            print("ROC VAL {0}".format(mean_roc))
            print("F1 {0}".format(avg_f1))
            print("KAPPA {0}".format(kappa_score))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/Colab Data/"

In [ ]:
global_cls = ['Normal', 'Pneumothorax']
global_cls = np.array(global_cls)

In [ ]:
X_train=np.load('/content/gdrive/My Drive/Colab Data/x_train_4000.npy', mmap_mode='r+')
Y_train=np.load('/content/gdrive/My Drive/Colab Data/y_train_final_classification_4000.npy', mmap_mode='r+')
X_test=np.load('/content/gdrive/My Drive/Colab Data/x_test_768_2.npy', mmap_mode='r+')
Y_test=np.load('/content/gdrive/My Drive/Colab Data/y_val_final_classification.npy', mmap_mode='r+')

In [ ]:
X_test.shape

In [ ]:
#@title Data preparation
y=[]
Y_train=np.load('/content/gdrive/My Drive/Colab Data/y_train_4000.npy', mmap_mode='r+')
for index, image in enumerate(Y_train):
  if np.max(image)==0:
    y.append(0)
  else:
    y.append(1)
  print (index)

from keras.utils import to_categorical
y=to_categorical(y)
y=np.array(y)

In [ ]:
roc_call = roc_callback((X_test,Y_test))

In [ ]:
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(768,768,3),pooling='avg')

In [ ]:
preds = Dense(2,activation='softmax')(dense_model.output)
model = Model(dense_model.input,preds)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1,
                              patience=4, min_lr=0.0000001, verbose=1)

In [ ]:
class_weights={0:1, 1:4}

In [ ]:
model_checkpoint = ModelCheckpoint(
        os.path.join('/content/gdrive/My Drive/Colab Data/Weights/', 'DenseNet121_classifier_lr_768_768_2.hdf5'),
        monitor='val_acc', mode='max',save_best_only=True, verbose=1)
lrate = LearningRateScheduler(step_decay)
callbacks = [model_checkpoint, roc_call, reduce_lr]

In [ ]:
model.fit(X_train, Y_train, verbose=1, validation_data=(X_test, Y_test),epochs=50, batch_size=2, callbacks=callbacks, shuffle=True)

In [ ]:
model.load_weights('/content/gdrive/My Drive/Colab Data/Weights/DenseNet121_classifier_lr_768_768.hdf5')

In [ ]:
y_pred=model.predict(X_test, verbose=1)

In [ ]:
Y_test_new=[np.argmax(i) for i in Y_test]
y_pred_new=[np.argmax(i) for i in y_pred]

In [ ]:
from sklearn.metrics import classification_report

Y_test_new=[np.argmax(i) for i in Y_test]
y_pred_new=[np.argmax(i) for i in y_pred]
    
print(classification_report(Y_test_new, y_pred_new, target_names=global_cls))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test_new, y_pred_new)

In [ ]:
man_y=[]

for i in y_pred:
  if i[1]>0.05:
    man_y.append(1)
  else:
    man_y.append(0)

In [ ]:
confusion_matrix(Y_test_new, man_y)